In [3]:
import nglview
import ipywidgets
import os
import zipfile

## Step 1: Protein and ligand selection

In [4]:
# DOI: 10.2210/pdb3HTB/pdb
# 2-propylphenol in complex with T4 lysozyme
pdbCode = "3HTB"

ligandCode = "JZ4"

mol_charge = 0

## Step 2: Downloading Protein

From: RCSB PDB database

In [5]:
from biobb_io.api.pdb import pdb

# Create properties dict and inputs/outputs
pdb_output = pdbCode+'.orig.pdb'

prop = {
    'pdb_code': pdbCode,
    'filter': False
}

# Downloading 3HTB from the Protein Data Bank
pdb(output_pdb_path=pdb_output, properties=prop)

2021-11-02 21:36:14,766 [MainThread  ] [INFO ]  Downloading: 3htb from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb3htb.ent


/home/dhnchandan/anaconda3/envs/nb_md_analysis/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ebi.ac.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


2021-11-02 21:36:17,580 [MainThread  ] [INFO ]  Writting pdb to: 3HTB.orig.pdb


0